# Just exploring some potential datasets

In [111]:
from ucimlrepo import fetch_ucirepo, list_available_datasets
from sklearn.preprocessing import LabelEncoder

#list_avaliable_datasets()


In [112]:
dataset_id = {"iris": 53, 
              "heart_disease": 45,
              "wine_quality": 186,
              "breast_cancer_wisconsin": 17,
              "car_evaluation": 19, # Not good maybe remove
              "abalone": 1,
              "mushroom": 73, # a lot of categorical data
              "statlog" : 144 # german credit
              }

In [113]:
#TODO Convert categorical to numerical with Label encoder
def import_dataset(uci_id):
    dataset = fetch_ucirepo(id=uci_id) 
    data_types = dataset.variables["type"]
    X = dataset.data.features 
    y = dataset.data.targets 
    for i, type in enumerate(data_types):
        print(type)
    return X, y


In [114]:
X, y = import_dataset(dataset_id["statlog"])
print(y)
print(X)

Categorical
Integer
Categorical
Categorical
Integer
Categorical
Categorical
Integer
Categorical
Categorical
Integer
Categorical
Integer
Categorical
Categorical
Integer
Categorical
Integer
Binary
Binary
Binary
     class
0        1
1        2
2        1
3        1
4        2
..     ...
995      1
996      1
997      1
998      2
999      1

[1000 rows x 1 columns]
    Attribute1  Attribute2 Attribute3 Attribute4  Attribute5 Attribute6  \
0          A11           6        A34        A43        1169        A65   
1          A12          48        A32        A43        5951        A61   
2          A14          12        A34        A46        2096        A61   
3          A11          42        A32        A42        7882        A61   
4          A11          24        A33        A40        4870        A61   
..         ...         ...        ...        ...         ...        ...   
995        A14          12        A32        A42        1736        A61   
996        A11          30        